## Query the Planet with EOLearn and Sentinel Data

The availability of open Earth observation (EO) data through the Copernicus and Landsat programs represents an unprecedented resource for many EO applications, ranging from ocean and land use and land cover monitoring, disaster control, emergency services and humanitarian relief. Given the large amount of high spatial resolution data at high revisit frequency, techniques able to automatically extract complex patterns in such spatio-temporal data are needed.

[`eo-learn`](https://eo-learn.readthedocs.io/en/latest/) is a collection of open source Python packages that have been developed to seamlessly access and process spatio-temporal image sequences acquired by any satellite fleet in a timely and automatic manner. eo-learn is easy to use, it’s design modular, and encourages collaboration – sharing and reusing of specific tasks in a typical EO-value-extraction workflows, such as cloud masking, image co-registration, feature extraction, classification, etc. Everyone is free to use any of the available tasks and is encouraged to improve them, develop new ones and share them with the rest of the community.

We'll walk through the basics of `eo-learn` and an example EO analysis workflow.

![eo-learn workflow](../images/eo-learn-illustration.png)

## Boring Setup

In [ ]:
# Jupyter notebook related commands
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# set our Sentinel Hub Instance ID (Workshop only)
# Sign-up at afterwards at https://www.sentinel-hub.com/ to continue using
!sentinelhub.config --instance_id insert_instance_id_here

## EO Learn Fundamentals

`eo-learn` has three fundamental classes `EOPatch`, `EOTask` and `EOWorkflow` which are helpful to understand before undertaking a larger project

### EOPatch
- an **abstract representation of remotely sensed data over a single area**; the base data class of `eo-learn`
- a collection of 10 python dicts representing various types of data along with a bounding box and timestamps
- provides useful dimension and type checking, a consistent interface for later processing, and some convenience functions

Find more details in the [documentation](https://eo-learn.readthedocs.io/en/latest/examples/core/CoreOverview.html#EOPatch)

In [ ]:
# first create an empty patch, it is instantiated with all the data types
# each FeatureType (data, mask, scalar, ...) requires a specific type of data
from eolearn.core import EOPatch

patch = EOPatch()
patch

In [ ]:
# the first Featuretype 'data' should be a numpy array with four dimensions, time x height x width x channel depth, to represent a time stack of remotely sensed data 

# create an example of satellite data with 5 dates, 10x10 pixels, and 3 bands
import numpy as np
example_bands = np.zeros((5, 10, 10, 3), dtype=np.uint16)
patch.data['example'] = example_bands

In [ ]:
# EOPatches will throw an error if you give data of the wrong shape to a certain FeatureType

# here's satellite data with no time component
timeless_data = np.zeros((10, 10, 13), dtype=np.uint16)
try:
    patch.data['timeless_example'] = timeless_data
except ValueError as e:
    print(e)

In [ ]:
# there is a different FeatureType available for data without a time component
patch.data_timeless['timeless_example'] = timeless_data

In [ ]:
# try to add data to other FeatureTypes, use https://eo-learn.readthedocs.io/en/latest/examples/core/CoreOverview.html#EOPatch for reference
import geopandas as gpd
from datetime import datetime
from sentinelhub import BBox, CRS

patch.mask['shadow masks'] = ...
patch.scalar['weather data'] = ...
patch.label['classification'] = ...
patch.vector['land use vectors'] = ...

patch.mask_timeless['district_boundaries_raster'] = ...
patch.scalar_timeless['single patch value'] = ...
patch.label_timeless['another patch value'] = ...
patch.vector_timeless['region_boundaries'] = ...

patch.meta_info = ...
patch.bbox = ...
patch.timestamp = ...

patch.get_features()

In [ ]:
# we can save and load our patch; check out the file structure it creates
patch.save('example_patch')
patch2 = patch.load('example_patch')
patch2.get_features()

### EOTask
- an **abstract representation of an operation over remotely sensed data** over a single area; the base processing class of `eo-learn`
- must have an `execute` method which generally accepts an `EOPatch` instance as its first argument
- provides consistency of operation, chainable, predictable outputs
- many `EOTasks` are already available in the library (60+ at the time of writing)

In [ ]:
# let's start with a simple one to understand the concept
# AddFeature performs the same operation we were doing before but in EOTask form
from eolearn.core import AddFeature, FeatureType

# specify the feature type and name, then instantiate the task (but not call it yet)
feature = (FeatureType.DATA, 'bands')
add_feature = AddFeature(feature)

# now we can execute the task, on our previous patch
data = np.zeros((5, 100, 100, 13))
patch = add_feature.execute(patch, data)
patch

In [ ]:
# now let's try a more substantial task
from eolearn.io import S2L1CWCSInput

# we'll make a new empty EOPatch over the conference for this
patch = EOPatch()
patch.bbox = BBox([26.08, 44.42, 26.14, 44.46], crs=CRS.WGS84)

# now we can load Sentinel-2 data into the patch
# first instantiate the task with a Sentinel Hub layer and our desired resolution
load_s2 = S2L1CWCSInput('TRUE-COLOR-S2-L1C', resx='10m', resy='10m')

# then call the task's execute method with our patch and a time interval
load_s2.execute(eopatch=patch, time_interval=('2019-08-23', '2019-08-27'))


In [ ]:
# since the data is a numpy array, let's try visualizing it with matplotlib
from matplotlib import pyplot as plt
plt.imshow(patch.data['TRUE-COLOR-S2-L1C'][0])

In [ ]:
# try with other resolutions, layers, date ranges or areas
# also notice the additional information that the task adds to mask, meta_info and timestamp

In [ ]:
# now let's add some vector data, load a local file containing buildings
buildings = gpd.read_file('bucharest_buildings.geojson')

In [ ]:
# we can add this file as either a vector (directly) or try transforming it to a raster
from eolearn.geometry import VectorToRaster
vtr_task = VectorToRaster(
    buildings,
    values=1,
    raster_shape=(FeatureType.DATA, 'TRUE-COLOR-S2-L1C'), # match the shape of our previous data
    raster_feature=(FeatureType.MASK_TIMELESS, 'BUILDING-DATA') # name our new layer
)

vtr_task.execute(eopatch=patch)

In [ ]:
# now look at the buildings (also congrats, you're 50% of the way to being a geospatial machine learning engineer)
plt.imshow(patch.mask_timeless['BUILDING-DATA'].squeeze(2))

In [ ]:
# we can also implement our own tasks, they need to:
# - inherit from the EOTask class
# - implement an __init__ method
# - implement an execute method which accepts an EOPatch as the first argument

# here's a template, try it out!
from eolearn.core import EOTask

# one suggestion is try to calculate the median pixel of a timeseries stack
class OurCustomTask(EOTask):
    def __init__(self, custom_param):
        self.custom_param = custom_param

    def execute(self, eopatch, *, patch_specific_param=None):
        # write code for out task and make changes to the input eopatch and return it
        return eopatch

In [ ]:
# finally let's save our patch
from eolearn.core import SaveToDisk, OverwritePermission

save = SaveToDisk('example', overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

save.execute(eopatch=patch, eopatch_folder='this_patch')

## EOWorkflow

- a **combination of `EOTask` objects** meant to accomplish a larger project
- builds a computational graph object to ensure all patches are processed in the same fashion

In [ ]:
# rather than call .execute directly, let chain our tasks together
from eolearn.core import LinearWorkflow

workflow = LinearWorkflow(load_s2, vtr_task, save)

In [ ]:
# now we can call this on any number of patches and pass the execution arguments to the workflow
patch = EOPatch()
bbox = BBox([26.08, 44.42, 26.14, 44.46], crs=CRS.WGS84)
patch.bbox = bbox
patches = [patch]

for patch in patches:
    workflow.execute({
        load_s2: { 'time_interval': ('2019-08-23', '2019-08-27'), 'bbox': bbox },
        vtr_task: {},
        save: { 'eopatch_folder': 'patch_idx' }
    }).eopatch()
    


## Practice

Take some time to experiment with different tasks and parameters before starting a larger workflow

# Water Monitoring

Let's try a larger workflow, "Measuring the water level of a Theewaterskloof Dam in South Africa", modified from the [`eo-learn` documentation](https://eo-learn.readthedocs.io/en/latest/examples/water-monitor/WaterMonitorWorkflow.html). There are more examples and documentation there you can check out after the workshop.

The outline of the workflow is:
- 1: Download all available Sentinel-2 satellite imagery of Theewaterskloof Dam from beginning of 2018 until today. We'll download `TRUE_COLOR` for nicer visualisations + `NDWI` for water detection
- 2: Clouds are very often obscuring the view of the ground. In order to correctly determine the water level of the dam all images with clouds need to be filtered out.
- 3: Apply adaptive thresholding to NDWI grayscale images
- 4: Extract water level from a comparison of measured water extent with the nominal one

In [ ]:
import shapely.wkt
# Use the local WKT file of the dam outline to create a buffered geometry
with open('theewaterskloof_dam_nominal.wkt', 'r') as f:
    dam_wkt = f.read()

dam_nominal = shapely.wkt.loads(dam_wkt)

# inflate the BBOX
inflate_bbox = 0.1
minx, miny, maxx, maxy = dam_nominal.bounds

delx = maxx - minx
dely = maxy - miny
minx = minx - delx * inflate_bbox
maxx = maxx + delx * inflate_bbox
miny = miny - dely * inflate_bbox
maxy = maxy + dely * inflate_bbox

dam_bbox = BBox([minx, miny, maxx, maxy], crs=CRS.WGS84)

dam_bbox.geometry - dam_nominal

## Intialize our EOTasks


In [ ]:
## Imagery

# add true color Sentinel-2 data
input_task = S2L1CWCSInput('TRUE-COLOR-S2-L1C', resx='20m', resy='20m', maxcc=0.5, instance_id=None)

# add NDWI index from Sentinel-2
add_ndwi = S2L1CWCSInput('NDWI')

In [ ]:
## Geometry

# load the dam boundary as a DataFrame
dam_gdf = gpd.GeoDataFrame(crs={'init':'epsg:4326'}, geometry=[dam_nominal])
dam_gdf.plot()

# convert to raster
add_nominal_water = VectorToRaster(dam_gdf, (FeatureType.MASK_TIMELESS, 'NOMINAL_WATER'), values=1,
                                   raster_shape=(FeatureType.MASK, 'IS_DATA'), raster_dtype=np.uint8)

In [ ]:
## Cloud and Valid Data Filtering
from eolearn.mask import AddCloudMaskTask, AddValidDataMaskTask, get_s2_pixel_cloud_detector
from eolearn.features import SimpleFilterTask

# add cloud classification
cloud_classifier = get_s2_pixel_cloud_detector(average_over=2, dilation_size=1, all_bands=False)
cloud_detection = AddCloudMaskTask(cloud_classifier, 'BANDS-S2CLOUDLESS', cm_size_y='160m', cm_size_x='160m', cmask_feature='CLM', cprobs_feature='CLP', instance_id=None)

# if the pixel is clear, we count it as valid
def calculate_valid_data_mask(eopatch):
    return np.logical_and(eopatch.mask['IS_DATA'].astype(np.bool),
                          np.logical_not(eopatch.mask['CLM'].astype(np.bool)))

add_valid_mask = AddValidDataMaskTask(predicate=calculate_valid_data_mask)

# calculate valid pixels per patch
def calculate_coverage(array):
    return 1.0 - np.count_nonzero(array) / np.size(array)

class AddValidDataCoverage(EOTask):

    def execute(self, eopatch):
        valid_data = eopatch.get_feature(FeatureType.MASK, 'VALID_DATA')
        time, height, width, channels = valid_data.shape

        coverage = np.apply_along_axis(calculate_coverage, 1, valid_data.reshape((time, height * width * channels)))

        eopatch.add_feature(FeatureType.SCALAR, 'COVERAGE', coverage[:, np.newaxis])
        return eopatch

add_coverage = AddValidDataCoverage()

# filter out scenes with too many clouds
class ValidDataCoveragePredicate:

    def __init__(self, threshold):
        self.threshold = threshold

    def __call__(self, array):
        return calculate_coverage(array) < self.threshold

remove_cloudy_scenes = SimpleFilterTask((FeatureType.MASK, 'VALID_DATA'), ValidDataCoveragePredicate(0.2))

In [ ]:
## Water Detection
from skimage.filters import threshold_otsu

class WaterDetector(EOTask):

    @staticmethod
    def detect_water(ndwi):
        """
        Very simple water detector based on Otsu thresholding method of NDWI.
        """
        otsu_thr = 1.0
        if len(np.unique(ndwi)) > 1:
            otsu_thr = threshold_otsu(ndwi)

        return ndwi > otsu_thr

    def execute(self, eopatch):
        water_masks = np.asarray([self.detect_water(ndwi[...,0]) for ndwi in eopatch.data['NDWI']])

        # we're only interested in the water within the dam borders
        water_masks = water_masks[...,np.newaxis] * eopatch.mask_timeless['NOMINAL_WATER']

        water_levels = np.asarray([np.count_nonzero(mask)/np.count_nonzero(eopatch.mask_timeless['NOMINAL_WATER'])
                                   for mask in water_masks])

        eopatch.add_feature(FeatureType.MASK, 'WATER_MASK', water_masks)
        eopatch.add_feature(FeatureType.SCALAR, 'WATER_LEVEL', water_levels[...,np.newaxis])

        return eopatch

water_detection = WaterDetector()

## Define our EOWorkflow

In [ ]:
workflow = LinearWorkflow(input_task, add_ndwi, cloud_detection, add_nominal_water, add_valid_mask, add_coverage, remove_cloudy_scenes, water_detection)

## Execute the workflow

In [ ]:
# this takes a while
result = workflow.execute({
    input_task: {
        'bbox': dam_bbox,
        'time_interval': ['2018-01-01','2019-08-31']
    },
})

In [ ]:
patch = result.eopatch()
patch

## Plot the results

In [ ]:
from skimage.filters import sobel
from skimage.morphology import disk
from skimage.morphology import erosion, dilation, opening, closing, white_tophat

def plot_rgb_w_water(eopatch, idx):
    ratio = np.abs(eopatch.bbox.max_x - eopatch.bbox.min_x) / np.abs(eopatch.bbox.max_y - eopatch.bbox.min_y)
    fig, ax = plt.subplots(figsize=(ratio * 10, 10))

    ax.imshow(eopatch.data['TRUE-COLOR-S2-L1C'][idx])

    observed = closing(eopatch.mask['WATER_MASK'][idx,...,0], disk(1))
    nominal = sobel(eopatch.mask_timeless['NOMINAL_WATER'][...,0])
    observed = sobel(observed)
    nominal = np.ma.masked_where(nominal == False, nominal)
    observed = np.ma.masked_where(observed == False, observed)

    ax.imshow(nominal, cmap=plt.cm.Reds)
    ax.imshow(observed, cmap=plt.cm.Blues)
    ax.axis('off')

In [ ]:
plot_rgb_w_water(patch, 0)

In [ ]:
plot_rgb_w_water(patch, -1)

In [ ]:
def plot_water_levels(eopatch, max_coverage=1.0):
    fig, ax = plt.subplots(figsize=(20, 7))

    dates = np.asarray(eopatch.timestamp)
    ax.plot(dates[eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            eopatch.scalar['WATER_LEVEL'][eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            'bo-', alpha=0.7)
    ax.plot(dates[eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            eopatch.scalar['COVERAGE'][eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            '--', color='gray', alpha=0.7)
    ax.set_ylim(0.0, 1.1)
    ax.set_xlabel('Date')
    ax.set_ylabel('Water level')
    ax.set_title('Theewaterskloof Dam Water Levels')
    ax.grid(axis='y')
    return ax

In [ ]:
ax = plot_water_levels(patch, 1.0) # parameter is cloud cover filter

In [ ]:
ax = plot_water_levels(patch, 0.02)